### 準備

1. [筆者らGitHub](https://github.com/gifale95/eeg_encoding)をまるごとダウンロードして自分のGoogle Driveの任意の場所に置く.

<p align='center'><img src='https://github.com/SeanNobel/img2eeg/blob/main/figs/pr1.png?raw=True' width='60%'></p>

2. `eeg_encoding-main`下に`project_dir/image_set`を作成し, そこに[データセットページ](https://osf.io/y63gw/)の`test_images.zip`をダウンロードして解凍したものを配置する. (ドライブの容量に余裕があれば`training_images.zip`やILSVRC-2012データセットも配置してみても良いと思います)

<p align='center'><img src='https://github.com/SeanNobel/img2eeg/blob/main/figs/pr2.png?raw=True' width='60%'></p>

In [3]:
work_dir = '/content/drive/MyDrive/eeg_encoding-main/'

import os
# Google DriveをColabにマウント
from google.colab import drive
# drive.flush_and_unmount()
drive.mount("/content/drive", force_remount=True)

os.chdir(work_dir)
print(os.getcwd())

# !pip install -r requirements.txt

# 要素にドットでアクセスできる辞書クラス
class Args(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__dict__ = self

Mounted at /content/drive
/content/drive/MyDrive/eeg_encoding-main


### AlexNetを使って画像からfeature mapsを抽出する

<p align='center'><img src='https://github.com/SeanNobel/img2eeg/blob/main/figs/fig2.png?raw=True' width='60%'></p>

下のセルを実行すると, `project_dir/dnn_feature_maps/full_feature_maps/alexnet/test_images`にfeature mapsが200個出力されているのがわかります.

`02_dnn_feature_maps_extraction`に, ResNet-50など他のDNNを使ってfeature mapsを抽出するコードもあるので, 試してみるのもありかと思います. (その際はコードの`#Input arguments`あたりを僕と同じように編集する必要があると思います.)

In [4]:
"""Extracting and saving the AlexNet feature maps of the training and test
images, and of the ILSVRC-2012 validation and test images.
Parameters
----------
project_dir : str
	Directory of the project folder.
"""

# import argparse
from torchvision import models
import torch.nn as nn
import numpy as np
import torch
from torch.autograd import Variable as V
from torchvision import transforms as trn
import os
from PIL import Image


# =============================================================================
# Input arguments
# =============================================================================
# parser = argparse.ArgumentParser()
# parser.add_argument('--project_dir', default='/project/directory', type=str)
# args = parser.parse_args()

args = Args({
    'project_dir': work_dir + 'project_dir'
})

print('Extracting feature maps AlexNet <<<')
print('\nInput arguments:')
# for key, val in vars(args).items():
# 	print('{:16} {}'.format(key, val))


# =============================================================================
# Selecting the layers of interest and importing the model
# =============================================================================
# Lists of AlexNet convolutional and fully connected layers
conv_layers = ['conv1', 'ReLU1', 'maxpool1', 'conv2', 'ReLU2', 'maxpool2',
	'conv3', 'ReLU3', 'conv4', 'ReLU4', 'conv5', 'ReLU5', 'maxpool5']
fully_connected_layers = ['Dropout6', 'fc6', 'ReLU6', 'Dropout7', 'fc7',
	'ReLU7', 'fc8']

class AlexNet(nn.Module):
	def __init__(self):
		"""Selecting the desired layers and importing pretrained weights."""
		super(AlexNet, self).__init__()
		self.select_cov = ['maxpool1', 'maxpool2', 'ReLU3', 'ReLU4', 'maxpool5']
		self.select_fully_connected = ['ReLU6' , 'ReLU7', 'fc8']
		self.feat_list = self.select_cov + self.select_fully_connected
		self.alex_feats = models.alexnet(pretrained=True).features
		self.alex_classifier = models.alexnet(pretrained=True).classifier
		self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

	def forward(self, x):
		"""Extracting the feature maps."""
		features = []
		for name, layer in self.alex_feats._modules.items():
			x = layer(x)
			if conv_layers[int(name)] in self.feat_list:
				features.append(x)
		x = self.avgpool(x)
		x = x.view(x.size(0), -1)
		for name, layer in self.alex_classifier._modules.items():
			x = layer(x)
			if fully_connected_layers[int(name)] in self.feat_list:
				features.append(x)
		return features

model = AlexNet()
if torch.cuda.is_available():
	model.cuda()
model.eval()


# =============================================================================
# Defining the image preprocessing
# =============================================================================
centre_crop = trn.Compose([
	trn.Resize((224,224)),
	trn.ToTensor(),
	trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# =============================================================================
# Loading the images and extracting the corresponding feature maps
# =============================================================================
# Extracting the feature maps of (1) training images, (2) test images,
# (3) ILSVRC-2012 validation images, (4) ILSVRC-2012 test images.
# Image directories
img_set_dir = os.path.join(args.project_dir, 'image_set')
img_partitions = os.listdir(img_set_dir)
for p in img_partitions:
	part_dir = os.path.join(img_set_dir, p)
	image_list = []
	for root, dirs, files in os.walk(part_dir):
		for file in files:
			if file.endswith(".jpg") or file.endswith(".JPEG"):
				image_list.append(os.path.join(root,file))
	image_list.sort()

	# Extracting and saving the feature maps
	idx = 1
	for image in image_list:
		img = Image.open(image).convert('RGB')
		filename=image.split("/")[-1].split(".")[0]
		input_img = V(centre_crop(img).unsqueeze(0))
		if torch.cuda.is_available():
			input_img=input_img.cuda()
		x = model.forward(input_img)
		feats = {}
		for i,feat in enumerate(x):
			feats[model.feat_list[i]] = feat.data.cpu().numpy()

		# Creating the directory if not existing and saving
		save_dir = os.path.join(args.project_dir, 'dnn_feature_maps',
			'full_feature_maps', 'alexnet', p)
		file_name = p + '_' + format(idx, '07')
		if os.path.isdir(save_dir) == False:
			os.makedirs(save_dir)
		np.save(os.path.join(save_dir, file_name), feats)
		idx += 1

Extracting feature maps AlexNet <<<

Input arguments:


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]